In [6]:
# This script explores CNN model performance based on training image resolution.
# We test the following training image sizes:
# 64 x 64, 128 x 128, 224 x 224, 384 x 384

import numpy as np
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import pydot
import random

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback

        
from sklearn.metrics import recall_score, classification_report
from sklearn.datasets import make_classification

import json

import sys
sys.path.append("../python/")
from helpers import *

In [26]:
images_size64_exp5_Pr_Po_Im = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_Po_Im.npy', allow_pickle=True)
images_size64_exp5_Pr_Im = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_Im.npy', allow_pickle=True)
images_size64_exp5_PrPo_Im = np.load('../../data/tidy/preprocessed_images/size64_exp5_PrPo_Im.npy', allow_pickle=True)
images_size64_exp5_Pr_PoIm = np.load('../../data/tidy/preprocessed_images/size64_exp5_Pr_PoIm.npy', allow_pickle=True)

images_size128_exp5_Pr_Po_Im = np.load('../../data/tidy/preprocessed_images/size128_exp5_Pr_Po_Im.npy', allow_pickle=True)
images_size128_exp5_Pr_Im = np.load('../../data/tidy/preprocessed_images/size128_exp5_Pr_Im.npy', allow_pickle=True)
images_size128_exp5_PrPo_Im = np.load('../../data/tidy/preprocessed_images/size128_exp5_PrPo_Im.npy', allow_pickle=True)
images_size128_exp5_Pr_PoIm = np.load('../../data/tidy/preprocessed_images/size128_exp5_Pr_PoIm.npy', allow_pickle=True)

images_size224_exp5_Pr_Po_Im = np.load('../../data/tidy/preprocessed_images/size224_exp5_Pr_Po_Im.npy', allow_pickle=True)
images_size224_exp5_Pr_Im = np.load('../../data/tidy/preprocessed_images/size224_exp5_Pr_Im.npy', allow_pickle=True)
images_size224_exp5_PrPo_Im = np.load('../../data/tidy/preprocessed_images/size224_exp5_PrPo_Im.npy', allow_pickle=True)
images_size224_exp5_Pr_PoIm = np.load('../../data/tidy/preprocessed_images/size224_exp5_Pr_PoIm.npy', allow_pickle=True)

images_size384_exp5_Pr_Po_Im = np.load('../../data/tidy/preprocessed_images/size384_exp5_Pr_Po_Im.npy', allow_pickle=True)
images_size384_exp5_Pr_Im = np.load('../../data/tidy/preprocessed_images/size384_exp5_Pr_Im.npy', allow_pickle=True)
images_size384_exp5_PrPo_Im = np.load('../../data/tidy/preprocessed_images/size384_exp5_PrPo_Im.npy', allow_pickle=True)
images_size384_exp5_Pr_PoIm = np.load('../../data/tidy/preprocessed_images/size384_exp5_Pr_PoIm.npy', allow_pickle=True)

In [27]:
NUM_CLASS = 2
NUM_CHANNELS = 1

In [41]:
input_image_shape_64 = getImageShape(images_size64_exp5_Pr_Im, num_channels = NUM_CHANNELS)
input_image_shape_128 = getImageShape(images_size128_exp5_Pr_Im, num_channels = NUM_CHANNELS)
input_image_shape_224 = getImageShape(images_size224_exp5_Pr_Im, num_channels = NUM_CHANNELS)
input_image_shape_384 = getImageShape(images_size384_exp5_Pr_Im, num_channels = NUM_CHANNELS)

In [65]:
# Can either load a previously saved model or define here.
# model = models.load_model('../../results/models/MODEL_NAME')
base_model_64 = models.Sequential([
    layers.Conv2D(filters = 64, kernel_size = 7, strides = 2, activation="relu", padding="same", input_shape = input_image_shape_64),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5), # randomly drop out 50% of the neuorns at each training step
    layers.Dense(64, activation="relu"), # flatten all outputs
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASS, activation="softmax")
])

base_model_128 = models.Sequential([
    layers.Conv2D(filters = 64, kernel_size = 7, strides = 2, activation="relu", padding="same", input_shape = input_image_shape_128),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5), # randomly drop out 50% of the neuorns at each training step
    layers.Dense(64, activation="relu"), # flatten all outputs
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASS, activation="softmax")
])

base_model_224 = models.Sequential([
    layers.Conv2D(filters = 64, kernel_size = 7, strides = 2, activation="relu", padding="same", input_shape = input_image_shape_224),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5), # randomly drop out 50% of the neuorns at each training step
    layers.Dense(64, activation="relu"), # flatten all outputs
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASS, activation="softmax")
])
base_model_384 = models.Sequential([
    layers.Conv2D(filters = 64, kernel_size = 7, strides = 2, activation="relu", padding="same", input_shape = input_image_shape_384),
    layers.MaxPooling2D(2),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.BatchNormalization(),
    layers.Dropout(0.5), # randomly drop out 50% of the neuorns at each training step
    layers.Dense(64, activation="relu"), # flatten all outputs
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASS, activation="softmax")
])
# Compile the model.
#opt = SGD(lr = 0.001) #default learning rate (lr) = 0.1
# opt = keras.optimizers.Adam(learning_rate=0.01)
#model.compile(loss='categorical_crossentropy',  optimizer = "adam", metrics=[precision,recall, f1, 'accuracy'])

In [139]:
# The resolution test will be conducted scenario by scenario.
def test_resolution_performance(scenario, model_list, num_epochs = 10, trial_seed = 1): 
    resolution_keys = [64, 128, 224, 384]
    performance_dict = dict.fromkeys(resolution_keys)
    if scenario == "Pr_Po_Im":
        imageset = [images_size64_exp5_Pr_Po_Im, images_size128_exp5_Pr_Po_Im, images_size224_exp5_Pr_Po_Im, images_size384_exp5_Pr_Po_Im]
    elif scenario == "Pr_Im":
        imageset = [images_size64_exp5_Pr_Im]#, images_size128_exp5_Pr_Im, images_size224_exp5_Pr_Im, images_size384_exp5_Pr_Im]
    elif scenario == "PrPo_Im":
        imageset = [images_size64_exp5_PrPo_Im, images_size128_exp5_PrPo_Im, images_size224_exp5_PrPo_Im, images_size384_exp5_PrPo_Im]
    elif scenario == "Pr_PoIm":
        imageset = [images_size64_exp5_Pr_PoIm, images_size128_exp5_Pr_PoIm, images_size224_exp5_Pr_PoIm, images_size384_exp5_Pr_PoIm]

    k = 0
    for resolution_set in imageset:
        res_key = resolution_keys[k]
        performance_dict[res_key] = {}
        
        training_images_and_labels, test_images_and_labels = splitData(resolution_set, prop = 0.80, seed_num = trial_seed)
        training_images, training_labels = getImageAndLabelArrays(training_images_and_labels)
        validation_images, validation_labels = getImageAndLabelArrays(test_images_and_labels)
        batch_training_histories = Histories()
        # metrics_multiclass = Metrics(validation_images,validation_labels)  TODO
        # K.clear_session()
        model = model_list[k] # select k'th model from list
        reset_weights(model) # re-initialize model weights
        model.compile(loss='categorical_crossentropy',  optimizer = "adam", metrics = ['accuracy'])
        hist = model.fit(training_images, training_labels, batch_size = 32, epochs = num_epochs, verbose=1, validation_data=(validation_images, validation_labels)) #, callbacks=[batch_training_histories])
        performance_dict[res_key]['scenario'] = scenario
        performance_dict[res_key]['image_size'] = res_key
        performance_dict[res_key]['metrics'] = hist
        k += 1    
    return(performance_dict)

In [ ]:
RESOLUTION_PERFORMANCE_METRICS_DIR = '../../results/resolution-tests'
def main(num_trials = 5):
    scenario_list = ["Pr_Po_Im", "Pr_Im", "PrPo_Im", "Pr_PoIm"]
    optimized_models = [base_model_64, base_model_128, base_model_224, base_model_384] # TO BE UPDATED
    scenario_performance_dict = dict.fromkeys(scenario_list)
    if not os.path.exists(RESOLUTION_PERFORMANCE_METRICS_DIR): # check if 'tidy/preprocessed_images' subdirectory does not exist
        os.makedirs(RESOLUTION_PERFORMANCE_METRICS_DIR) # if not, create it    
    for i in range(num_trials):
        for j in scenario_list:
            scenario_performance_dict[j] = test_resolution_performance(j, optimized_models, num_epochs = 10, trial_seed = 1 + i) #ultimately should be averaged across trials       
        scenario_filename = "resolution_performance_" + j + "_trial_" + str(i) + ".txt"
        with open(os.path.join(RESOLUTION_PERFORMANCE_METRICS_DIR, data_filename), 'w') as file:
            file.write(json.dumps(scenario_performance_dict)) # use `json.loads` to do the reverse
    return

In [ ]:
# Plotting 
# p1[64]['metrics'].history['loss']
# plt.close('all')
# for m in ['recall', 'precision', 'f1-score']:
#     for c in [0,1,2]:
#         plt.plot(metrics_multiclass.get(m,c), label='Class {0} {1}'.format(c,m))
        
# plt.legend(loc='lower right')
# plt.show()